In [2]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

model = SamModel.from_pretrained("facebook/sam-vit-base")
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]] # 2D localization of a window
#inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to("cuda")
outputs = model(**inputs)
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
scores = outputs.iou_scores


config.json:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

RuntimeError: PyTorch is not linked with support for cuda devices

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, GlobalAveragePooling2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import numpy as np

# Parameters
n_samples = 1000
n_frames = 4
frame_height, frame_width = 224, 224
input_shape = (n_frames, frame_height, frame_width, 3)

# Dummy dataset generation (replace with actual data loading)
# X is shape (1000, 4, 224, 224, 3), y is shape (1000,)
X = np.random.rand(n_samples, n_frames, frame_height, frame_width, 3).astype(np.float32)
y = np.random.randint(2, size=(n_samples,))

# SAM: Shift-and-Mix function
def shift_and_mix(features, shifts=4):
    shifted_features = []
    for i in range(shifts):
        shifted = tf.roll(features, shift=i, axis=1)
        shifted_features.append(shifted)
    mixed_features = tf.reduce_mean(tf.stack(shifted_features), axis=0)
    return mixed_features

# MAMBA: Memory-Augmented Multi-step Attention function
def memory_augmented_attention(features, memory_size=256):
    memory = tf.Variable(tf.zeros([memory_size]), trainable=False)
    attention_weights = tf.nn.softmax(tf.matmul(features, tf.expand_dims(memory, axis=-1)), axis=1)
    attended_features = tf.reduce_sum(attention_weights * features, axis=1)
    memory.assign(attended_features)
    return attended_features

# Feature extractor using ResNet50
cnn_base = ResNet50(weights='imagenet', include_top=False, input_shape=(frame_height, frame_width, 3))
cnn_output = GlobalAveragePooling2D()(cnn_base.output)
cnn_model = Model(inputs=cnn_base.input, outputs=cnn_output)

# Model definition
video_input = Input(shape=input_shape)

# Apply CNN to each frame
time_distributed_cnn = TimeDistributed(cnn_model)(video_input)

# Apply SAM to the extracted features
sam_features = Lambda(shift_and_mix)(time_distributed_cnn)


# LSTM layer to process the sequence of frames
lstm_output = LSTM(256, return_sequences=False)(mamba_features)

# Fully connected layer
fc = Dense(128, activation='relu')(lstm_output)
output = Dense(1, activation='sigmoid')(fc)

# Final model
model = Model(inputs=video_input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)